### 1. Import dependencies

In [ ]:
!pip install tensorflow==2.7.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [1]:
from gym.spaces import Discrete, Box
import numpy as np
import matplotlib.pyplot as plt
import random

from BinPackingEnvironment1D import BinPacking

### 2. Create environment

In [2]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [3]:
env.action_space.sample()

8

In [4]:
env.observation_space.sample()

array([ 2, 10, 10,  1, 13, 20, 19,  5, 20, 19, 19])

### 3. Run baseline test (No ML)

In [5]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-5038
{'placed': 50, 'misplaced': 866, 'discarded': 84}
Episode:2 Score:-3997
{'placed': 48, 'misplaced': 861, 'discarded': 91}
Episode:3 Score:-4161
{'placed': 45, 'misplaced': 862, 'discarded': 93}
Episode:4 Score:-5158
{'placed': 50, 'misplaced': 865, 'discarded': 85}
Episode:5 Score:-3985
{'placed': 49, 'misplaced': 868, 'discarded': 83}
Episode:6 Score:-4679
{'placed': 46, 'misplaced': 868, 'discarded': 86}
Episode:7 Score:-4311
{'placed': 47, 'misplaced': 845, 'discarded': 108}
Episode:8 Score:-4474
{'placed': 53, 'misplaced': 850, 'discarded': 97}
Episode:9 Score:-4347
{'placed': 54, 'misplaced': 860, 'discarded': 86}
Episode:10 Score:-4418
{'placed': 49, 'misplaced': 863, 'discarded': 88}


### 4. Train an RL Model

In [6]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [7]:
# Will throw an error if these don't exist
log_path = os.path.join('Training', 'Logs')

In [8]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)


In [9]:
print(env.logs)

{'placed': 0, 'misplaced': 0, 'discarded': 0}


In [10]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [11]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_21
-----------------------------
| time/              |      |
|    fps             | 294  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 406         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008406086 |
|    clip_fraction        | 0.0706      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.39       |
|    explained_variance   | -0.000849   |
|    learning_rate        | 0.0003      |
|    loss                 | 2.24e+03    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0108     |
|    value_loss           | 5.86e+03    |
-----------------------------------------
--

### 5. Save model

In [12]:
PPO_Path = os.path.join('Training', 'Saved Models', 'Linear_PPO_Model')

In [13]:
model.save(PPO_Path)

### 6. Load model

In [ ]:
model = PPO.load(PPO_Path, env=env)

### 7. Test model

In [15]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    obs = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-1043
{'placed': 52, 'misplaced': 737, 'discarded': 211}
Episode:2 Score:-981
{'placed': 58, 'misplaced': 824, 'discarded': 118}
Episode:3 Score:-976
{'placed': 46, 'misplaced': 767, 'discarded': 187}
Episode:4 Score:-951
{'placed': 53, 'misplaced': 731, 'discarded': 216}
Episode:5 Score:-1000
{'placed': 56, 'misplaced': 772, 'discarded': 172}
Episode:6 Score:-991
{'placed': 45, 'misplaced': 785, 'discarded': 170}
Episode:7 Score:-1072
{'placed': 47, 'misplaced': 790, 'discarded': 163}
Episode:8 Score:-962
{'placed': 48, 'misplaced': 760, 'discarded': 192}
Episode:9 Score:-1054
{'placed': 47, 'misplaced': 730, 'discarded': 223}
Episode:10 Score:-1024
{'placed': 50, 'misplaced': 796, 'discarded': 154}


In [ ]:
print(env.metadata)

In [ ]:
env.metadata = { 'placed':0, 'misplaced':0, 'discarded':0 }